<a href="https://colab.research.google.com/github/Seng-Pan/atomproject/blob/main/yolov8detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics kaggle roboflow pyyaml

In [ ]:
!pip install opendatasets

In [ ]:
!pip install torch torchvision

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/prathumarikeri/american-sign-language-09az")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: seng285
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/prathumarikeri/american-sign-language-09az


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
dataset_path = '/content/american-sign-language-09az/American'
output_path = '/content/datasets/asl'
os.makedirs(os.path.join(output_path, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'labels/val'), exist_ok=True)
# print(os.listdir(dataset_path))

### Data preprocessing

In [ ]:
import random
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image  # Import PIL for image resizing

# hyperparameters
target_size = (224, 224)
batch_size = 8
epochs = 10
workers = 8               # workers for data loading
validation_split = 0.15

#Optimize CPU Performance
os.environ["OMP_NUM_THREADS"] = str(workers)
os.environ["MKL_NUM_THREADS"] = str(workers)
os.environ["KMP_AFFINITY"] = "granularity=fine,compact,1,0"
os.environ["KMP_BLOCKTIME"] = "1"

# Create directories
os.makedirs(os.path.join(output_path, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_path, 'labels/val'), exist_ok=True)

# Class mapping
classes = sorted(os.listdir(dataset_path))
class_to_id = {cls: idx for idx, cls in enumerate(classes)}

for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    images = [img for img in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, img))]

    # Skip if no images are found
    if not images:
        print(f"Skipping '{cls}' because no images were found.")
        continue

    # Limit the number of images per class
    if len(images) > 500:  # Further reduce dataset size
        images = random.sample(images, 500)

    # Split into training and validation sets
    train_imgs, val_imgs = train_test_split(images, test_size=validation_split, random_state=42)

    # Process training images
    for img in train_imgs:
        img_path = os.path.join(cls_path, img)
        image = Image.open(img_path)
        image = image.resize(target_size)  # Resize image
        resized_img_path = os.path.join(output_path, 'images/train', img)
        image.save(resized_img_path)
        # Create label file
        label_path = os.path.join(output_path, 'labels/train', os.path.splitext(img)[0] + '.txt')
        with open(label_path, 'w') as f:
            f.write(f'{class_to_id[cls]} 0.5 0.5 1.0 1.0')

    # Process validation images
    for img in val_imgs:
        img_path = os.path.join(cls_path, img)
        image = Image.open(img_path)
        image = image.resize(target_size)  # Resize image
        resized_img_path = os.path.join(output_path, 'images/val', img)
        image.save(resized_img_path)
        # Create label file
        label_path = os.path.join(output_path, 'labels/val', os.path.splitext(img)[0] + '.txt')
        with open(label_path, 'w') as f:
            f.write(f'{class_to_id[cls]} 0.5 0.5 1.0 1.0')

# Create dataset.yaml
yaml_content = f"""
path: {output_path}
train: images/train
val: images/val

nc: {len(classes)}
names: {classes}
"""
with open(os.path.join(output_path, 'asl.pt'), 'w') as f:
    f.write(yaml_content)

print("Dataset processing complete!")

Dataset processing complete!


In [ ]:
import torch

# Check if GPU is available
def check_gpu():
    use_gpu = torch.cuda.is_available()
    unit = "cpu"

    if use_gpu:
        print("-------GPU Mode--------")
        unit = "cuda"
    else:
        print("-------CPU Mode--------")

    device = torch.device(unit)
    return device

device = check_gpu()

-------CPU Mode--------


#### modeling

In [ ]:
# train yolov8 model

from ultralytics import YOLO

# Load the YOLOv8n model
model = YOLO('yolov8n.pt')

# Freeze initial layers to reduce computation
for idx, param in enumerate(model.model.parameters()):
    if idx < 100:  # Freeze first 100 layers
        param.requires_grad = False

# Custom training loop with validation frequency
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Train for one epoch
    model.train(
        data=os.path.join(output_path, 'asl.pt'),
        epochs=1,
        imgsz=target_size[0],  # Match target_size
        batch=batch_size,
        name='asl_yolov8n_optimized',
        device=device,
        workers=workers,
        augment=False,
        single_cls=False,
        lr0=0.01,       # inital learning rate  (fine-tune)
        lrf=0.1,        # final lr  (f-t)
        patience=5,     # f-t
        weight_decay=0.0005  # Regularization   (f-t)
    )

    # Validate every 3 epochs
    if (epoch + 1) % 3 == 0:
        print("Running validation...")
        model.val()

# Save model
model.export(format='onnx')  # Export to ONNX format
print("Model training and validation complete!")